In [9]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
%matplotlib inline

from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import argparse

/Users/dima/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Read all the files

data_folder = "input_data/"
proc_data_folder = "processed_data/"


aisles = pd.read_csv(os.path.join(data_folder, "aisles.csv"))
departments = pd.read_csv(os.path.join(data_folder, "departments.csv"))
products = pd.read_csv(os.path.join(data_folder, "products.csv"))


# Sample from the big files for now
order_prod_prior = pd.read_csv(os.path.join(data_folder, "order_products__prior.csv"))
order_prod_train = pd.read_csv(os.path.join(data_folder, "order_products__train.csv"))
orders = pd.read_csv(os.path.join(data_folder, "orders.csv"))


In [3]:
# Sample 20 users' orders, inner join on order_id with order_prod_prior 
order_samp = orders[orders["user_id"].isin(range(1,21))]
train_subset = order_prod_prior.merge(order_samp, on="order_id")
target_subset = order_prod_train.merge(order_samp, on="order_id")

In [4]:
print(train_subset.shape, target_subset.shape)

(2045, 10) (137, 10)


In [7]:
# Generate sequences of products (discard other features for now)
# Can do this for each feature and build a matrix for each user (next iteration?)

train_seqs = np.array(train_subset
             .sort_values(["user_id","order_number","add_to_cart_order"])
             .groupby("user_id")["product_id"]
             .apply(list)
             )

target_seqs = np.array(target_subset
             .sort_values(["user_id","order_number","add_to_cart_order"])
             .groupby("user_id")["product_id"]
             .apply(list)
             )

print(train_seqs[0], target_seqs[0])

[196, 14084, 12427, 26088, 26405, 196, 10258, 12427, 13176, 26088, 13032, 196, 12427, 10258, 25133, 30450, 196, 12427, 10258, 25133, 26405, 196, 12427, 10258, 25133, 10326, 17122, 41787, 13176, 196, 12427, 10258, 25133, 196, 10258, 12427, 25133, 13032, 12427, 196, 10258, 25133, 46149, 49235, 49235, 46149, 25133, 196, 10258, 12427, 196, 46149, 39657, 38928, 25133, 10258, 35951, 13032, 12427] [196, 25133, 38928, 26405, 39657, 10258, 13032, 26088, 27845, 49235, 46149]


In [14]:
train_path = train_seqs
# valid_path = os.path.join(data_path, "ptb.valid.txt")
# test_path = os.path.join(data_path, "ptb.test.txt")

# build the complete vocabulary, then convert text data to list of integers
word_to_id = build_vocab(train_path)
train_data = train_seqs
valid_data = file_to_word_ids(valid_path, word_to_id)
test_data = file_to_word_ids(test_path, word_to_id)
vocabulary = len(word_to_id)
reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

print(train_data[:5])
#     print(word_to_id)
print(vocabulary)
print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))

NameError: name 'build_vocab' is not defined

In [13]:
word_to_id

{196: 0,
 10258: 2,
 10326: 10,
 12427: 1,
 13032: 4,
 13176: 6,
 14084: 11,
 17122: 12,
 25133: 3,
 26088: 7,
 26405: 8,
 30450: 13,
 35951: 14,
 38928: 15,
 39657: 16,
 41787: 17,
 46149: 5,
 49235: 9}

In [ ]:
def read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", "<eos>").split()


def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]


def load_data():
    # get the data paths
    train_path = os.path.join(data_path, "ptb.train.txt")
    valid_path = os.path.join(data_path, "ptb.valid.txt")
    test_path = os.path.join(data_path, "ptb.test.txt")

    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    valid_data = file_to_word_ids(valid_path, word_to_id)
    test_data = file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(train_data[:5])
#     print(word_to_id)
    print(vocabulary)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, valid_data, test_data, vocabulary, reversed_dictionary

train_data, valid_data, test_data, vocabulary, reversed_dictionary = load_data()